<a href="https://colab.research.google.com/github/tom1999-303/Metamorphic_test/blob/main/Metamorphic_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MNISTのデータセットで学習させたCNNに対してMetamorphic testingを試みる

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


from tensorflow.keras import datasets, layers, models

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
train_images=train_images.reshape((60000, 28, 28, 1))
test_images=test_images.reshape(((10000, 28, 28, 1)))
train_images, test_images = train_images / 255.0, test_images / 255.0

In [9]:
model= models.Sequential([
        layers.Conv2D(32,(3,3),activation="relu",input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(16,(3,3),activation="relu",input_shape=(13,13, 32)),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(10, activation='softmax')
        ])

In [10]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 16)        4624      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 400)               0         
                                                                 
 dense_2 (Dense)             (None, 10)               

In [11]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
              )

In [12]:
model.fit(train_images,train_labels,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 17s 5ms/step - loss: 0.2217 - accuracy: 0.9340
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0800 - accuracy: 0.9751
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0606 - accuracy: 0.9814
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0490 - accuracy: 0.9848
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0420 - accuracy: 0.9869


In [13]:
model.evaluate(test_images,test_labels,verbose=2)

313/313 - 1s - loss: 0.0399 - accuracy: 0.9872 - 1s/epoch - 4ms/step


[0.039884187281131744, 0.9872000217437744]

Metamorophic testで画像データに定数と加えて予測値が不変であることを確かめる．

In [14]:
metatest_images= test_images
metatest_labels=test_labels

In [15]:
def shifting_data(image,constant):
    image =np.clip(image+constant,0,1)
    return image

In [17]:
for t in  range(0, 10, 1):
    shifting_metatest_images = np.array([shifting_data(image,t/10) for image in test_images])
    predictions = model.predict(test_images)
    meta_predictions = model.predict(shifting_metatest_images)
    label_predictions = np.argmax(predictions,axis=1)
    label_meta_predictions = np.argmax(meta_predictions,axis=1)
    print(t/10,(len([i for i in range(10000) if label_predictions[i]==label_meta_predictions[i]])/10000)*100)

0.0 100.0
0.1 99.25
0.2 98.14
0.3 96.25
0.4 92.56
0.5 85.11
0.6 71.61999999999999
0.7 57.26
0.8 38.93
0.9 13.780000000000001
